In [1]:
# Process OBS so it's ready for comparison

In [2]:
import pandas as pd

# Load and Clean: new file version

In [8]:
# Read latest OBS data
df = pd.read_csv(r'R:\FastTrips\PathChoice_Validation\OBS_to_FToutput\OBS_FToutput.csv')

# Pull the person ID from the original record file (separated by "---"
df['person_id'] = df['person_id'].apply(lambda x: x.split("---")[0]).astype('str')

# Create empty fields we might populate later
df['A_id'] = df['A_id_num']
df['B_id'] = df['B_id_num']

# Fill NaN type with empty string
for field in ['A_id_num','B_id_num','A_id','B_id']:
    df[field].fillna("",inplace=True)
    
# The trip_list_id num are off in our input
# We need to make them match the fast trips outputs
# Lookup the trip_list_id_num from the input file; these are based on the trip otaz and dtaz

# Create a field designating the origin and destination taz for entire path
origin = pd.DataFrame(df.groupby(['person_id','trip_list_id_num']).first()['A_id_num'])
dest = pd.DataFrame(df.groupby(['person_id','trip_list_id_num']).last()['B_id_num'])

origin['path_o_taz'] = origin['A_id_num']
origin.drop('A_id_num',axis=1,inplace=True)
origin['person_id'] = origin.index.get_level_values(0)
origin['trip_list_id_num'] = origin.index.get_level_values(1)

dest['path_d_taz'] = dest['B_id_num']
dest.drop('B_id_num',axis=1,inplace=True)
dest['person_id'] = dest.index.get_level_values(0)
dest['trip_list_id_num'] = dest.index.get_level_values(1)

# Merge to include a path_origin and path_dest field for each path link
df = pd.merge(df, origin, on=['person_id','trip_list_id_num'],how='left')
df = pd.merge(df, dest, on=['person_id','trip_list_id_num'],how='left')

In [9]:
# Read in an Fast Trips input format
obsinput = pd.read_csv('../data/obs/trip_list.txt')
obsinput['person_id'] = obsinput['person_id'].astype('int').astype('str')
obsinput['trip_list_id_num'] = obsinput['trip_list_id_num'].astype('int').astype('str')

obsinput['unique_id'] = obsinput['person_id'] + "_" + obsinput['trip_list_id_num']

In [10]:
# Drop the old trip_list_id_num and replace by merging with original input values 
df.drop('trip_list_id_num',axis=1,inplace=True)
df = pd.merge(df,obsinput[['person_id','trip_list_id_num','o_taz','d_taz']],
              left_on=['person_id','path_o_taz','path_d_taz'], right_on=['person_id','o_taz','d_taz'], 
              how='inner')    
# Using inner join returns only person_id + trip_list_id_num pairs in the actual trip_list.txt file

In [11]:
df.to_csv(r'C:\Users\Brice\fast-trips-validation\data\obs\updated_obs\obs_links.csv',index=False)